# Batch 5 DEXA Data Cleaning

**Objective**: Extract and organize DEXA scan data from Batch 5 text files

**Data Source**: /Sample Data/DEXA Scans/Batch 5

**Output**: Organized Excel file with all measurements by subject, gender, and timepoint

**Process**: Scan → Parse → Organize → Export

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from pathlib import Path
import re

# Data paths
batch5_path = Path("/Users/aviado/Documents/GDG WashU Medicine/Sample Data/DEXA Scans/Batch 5")
output_dir = Path("../../../cleaned_output")
output_dir.mkdir(exist_ok=True)

## Data Scanning

In [2]:
# Scan all txt files in Batch 5
def scan_batch5_files():
    txt_files = []
    
    # Define timepoint directories for Batch 5
    timepoints = {
        'Pre-Scan': 'Pre_Scan',
        '1 week post-treatment': 'Week_1',
        '2 week post-treatment': 'Week_2', 
        '3 week post-treatment': 'Week_3',
        'Post-Scan': 'Post_Scan'
    }
    
    # Scan each timepoint directory
    for timepoint_dir, timepoint_name in timepoints.items():
        timepoint_path = batch5_path / timepoint_dir
        if timepoint_path.exists():
            # Check Male and Female subdirectories
            for gender in ['Male', 'Female']:
                gender_path = timepoint_path / gender
                if gender_path.exists():
                    # Get all txt files
                    for txt_file in gender_path.glob('*.txt'):
                        txt_files.append({
                            'file_path': txt_file,
                            'timepoint': timepoint_name,
                            'gender': gender,
                            'filename': txt_file.name
                        })
    
    # Also check root directory for any txt files
    for txt_file in batch5_path.glob('*.txt'):
        txt_files.append({
            'file_path': txt_file,
            'timepoint': 'Root',
            'gender': 'Unknown',
            'filename': txt_file.name
        })
    
    return txt_files

# Scan files
batch5_files = scan_batch5_files()
print(f"Found {len(batch5_files)} txt files in Batch 5")

# Show sample of found files
for i, file_info in enumerate(batch5_files[:5]):
    print(f"{i+1}. {file_info['timepoint']} - {file_info['gender']} - {file_info['filename']}")
if len(batch5_files) > 5:
    print(f"... and {len(batch5_files) - 5} more files")

Found 50 txt files in Batch 5
1. Pre_Scan - Male - B8M5.txt
2. Pre_Scan - Male - B8M4.txt
3. Pre_Scan - Male - B8M0.txt
4. Pre_Scan - Male - B8M1.txt
5. Pre_Scan - Male - B8M3.txt
... and 45 more files


## Data Parsing

In [3]:
# Parse DEXA txt file content
def parse_dexa_txt(file_path):
    """Extract measurements from DEXA txt file"""
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        
        # Extract subject ID from filename (adapt for Batch 5 naming)
        filename = file_path.name
        subject_match = re.search(r'(B5_[MF]_\d+)', filename)
        if not subject_match:
            # Try alternative patterns for Batch 5
            subject_match = re.search(r'([MF]\d+)', filename)
        subject_id = subject_match.group(1) if subject_match else filename.replace('.txt', '')
        
        # Extract measurements using regex patterns
        measurements = {'subject_id': subject_id}
        
        # Find WHOLE TISSUE STATISTICS section (more comprehensive data)
        whole_section = re.search(r'WHOLE TISSUE STATISTICS:(.*?)(?=\n\s*-|$)', content, re.DOTALL)
        if whole_section:
            section_text = whole_section.group(1)
        else:
            # Fallback to INSIDE ROI if WHOLE not found
            section_text = content
        
        # Define patterns for key measurements
        patterns = {
            'sample_area': r'Sample Area:\s*([\d.]+)\s*cm',
            'bone_area': r'Bone Area:\s*([\d.]+)\s*cm',
            'total_weight': r'Total Weight:\s*([\d.]+)\s*g',
            'soft_weight': r'Soft Weight:\s*([\d.]+)\s*g',
            'lean_weight': r'Lean Weight:\s*([\d.]+)\s*g',
            'fat_weight': r'Fat Weight:\s*([\d.]+)\s*g',
            'fat_percent': r'Fat Percent:\s*([\d.]+)',
            'bmc': r'BMC:\s*([\d.]+)\s*g',
            'bmd': r'BMD:\s*([\d.]+)\s*mg/cm'
        }
        
        # Extract each measurement
        for key, pattern in patterns.items():
            match = re.search(pattern, section_text)
            if match:
                measurements[key] = float(match.group(1))
            else:
                measurements[key] = None
        
        return measurements
        
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
        return None

# Test parsing with first file
if batch5_files:
    sample_file = batch5_files[0]['file_path']
    sample_data = parse_dexa_txt(sample_file)
    print(f"Sample parsing result:")
    print(f"File: {sample_file.name}")
    for key, value in sample_data.items():
        print(f"  {key}: {value}")

Sample parsing result:
File: B8M5.txt
  subject_id: M5
  sample_area: 33.523
  bone_area: 11.435
  total_weight: 34.0742
  soft_weight: 32.8412
  lean_weight: 22.4615
  fat_weight: 10.3796
  fat_percent: 31.606
  bmc: 1.23304
  bmd: 107.831


In [4]:
# Process all Batch 5 files
def process_all_batch5_files():
    all_data = []
    
    for file_info in batch5_files:
        # Parse the txt file
        measurements = parse_dexa_txt(file_info['file_path'])
        
        if measurements:
            # Add metadata
            measurements.update({
                'batch': 'Batch_5',
                'timepoint': file_info['timepoint'], 
                'gender': file_info['gender'],
                'filename': file_info['filename']
            })
            all_data.append(measurements)
    
    return pd.DataFrame(all_data)

# Process all files
batch5_df = process_all_batch5_files()

print(f"Processed {len(batch5_df)} files")
print(f"Columns: {list(batch5_df.columns)}")
print(f"Shape: {batch5_df.shape}")

# Show sample data
batch5_df.head()

Processed 50 files
Columns: ['subject_id', 'sample_area', 'bone_area', 'total_weight', 'soft_weight', 'lean_weight', 'fat_weight', 'fat_percent', 'bmc', 'bmd', 'batch', 'timepoint', 'gender', 'filename']
Shape: (50, 14)


,subject_id,sample_area,bone_area,total_weight,soft_weight,lean_weight,fat_weight,fat_percent,bmc,bmd,batch,timepoint,gender,filename
0,M5,33.523,11.435,34.0742,32.8412,22.4615,10.3796,31.606,1.23304,107.831,Batch_5,Pre_Scan,Male,B8M5.txt
1,M4,33.005,12.317,34.0905,32.6383,22.9954,9.6428,29.545,1.45228,117.908,Batch_5,Pre_Scan,Male,B8M4.txt
2,M0,37.423,12.678,40.0617,38.7079,28.1945,10.5133,27.161,1.35384,106.786,Batch_5,Pre_Scan,Male,B8M0.txt
3,M1,36.142,11.126,39.6344,38.4868,30.1166,8.3702,21.748,1.14761,103.150,Batch_5,Pre_Scan,Male,B8M1.txt
4,M3,32.174,10.797,32.9502,31.8330,22.5268,9.3062,29.234,1.11723,103.480,Batch_5,Pre_Scan,Male,B8M3.txt


## Data Analysis

In [5]:
# Analyze the organized data
print("Batch 5 Data Summary:")
print(f"Total subjects: {batch5_df['subject_id'].nunique()}")
print(f"Timepoints: {batch5_df['timepoint'].unique()}")
print(f"Gender distribution: {batch5_df['gender'].value_counts().to_dict()}")

# Check for missing values
missing_data = batch5_df.isnull().sum()
numeric_cols = batch5_df.select_dtypes(include=[np.number]).columns
print(f"\nMissing values in key measurements:")
for col in numeric_cols:
    if missing_data[col] > 0:
        print(f"  {col}: {missing_data[col]} missing")

# Subject tracking across timepoints
subject_timepoints = batch5_df.groupby('subject_id')['timepoint'].nunique().sort_values(ascending=False)
print(f"\nSubject longitudinal tracking:")
print(f"Subjects with multiple timepoints: {len(subject_timepoints[subject_timepoints > 1])}")
print(f"Most tracked subject has {subject_timepoints.iloc[0]} timepoints")

# Show distribution by timepoint and gender
timepoint_gender = batch5_df.groupby(['timepoint', 'gender']).size().unstack(fill_value=0)
print(f"\nScans by timepoint and gender:")
print(timepoint_gender)

Batch 5 Data Summary:
Total subjects: 10
Timepoints: ['Pre_Scan' 'Week_1' 'Week_2' 'Week_3' 'Post_Scan']
Gender distribution: {'Male': 30, 'Female': 20}

Missing values in key measurements:

Subject longitudinal tracking:
Subjects with multiple timepoints: 10
Most tracked subject has 5 timepoints

Scans by timepoint and gender:
gender     Female  Male
timepoint              
Post_Scan       4     6
Pre_Scan        4     6
Week_1          4     6
Week_2          4     6
Week_3          4     6


## Data Cleaning

In [6]:
# Clean the Batch 5 dataset
def clean_batch5_data(df):
    df_clean = df.copy()
    
    # Fill missing numeric values with median (more appropriate for DEXA measurements)
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_clean[col].isnull().sum() > 0:
            if df_clean[col].notna().sum() > 0:
                median_val = df_clean[col].median()
                df_clean[col] = df_clean[col].fillna(median_val)
            else:
                df_clean[col] = df_clean[col].fillna(0)
    
    # Fill missing categorical values
    categorical_cols = df_clean.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col] = df_clean[col].fillna("Unknown")
    
    # Organize columns in logical order
    column_order = [
        'batch', 'subject_id', 'timepoint', 'gender', 'filename',
        'total_weight', 'soft_weight', 'lean_weight', 'fat_weight', 'fat_percent',
        'bmc', 'bmd', 'bone_area', 'sample_area'
    ]
    
    # Reorder columns (keep any extra columns at the end)
    available_cols = [col for col in column_order if col in df_clean.columns]
    extra_cols = [col for col in df_clean.columns if col not in column_order]
    df_clean = df_clean[available_cols + extra_cols]
    
    return df_clean

# Clean the data
batch5_cleaned = clean_batch5_data(batch5_df)

print(f"Data cleaning complete")
print(f"Missing values remaining: {batch5_cleaned.isnull().sum().sum()}")
print(f"Final shape: {batch5_cleaned.shape}")

# Show cleaned data sample
batch5_cleaned.head()

Data cleaning complete
Missing values remaining: 0
Final shape: (50, 14)


,batch,subject_id,timepoint,gender,filename,total_weight,soft_weight,lean_weight,fat_weight,fat_percent,bmc,bmd,bone_area,sample_area
0,Batch_5,M5,Pre_Scan,Male,B8M5.txt,34.0742,32.8412,22.4615,10.3796,31.606,1.23304,107.831,11.435,33.523
1,Batch_5,M4,Pre_Scan,Male,B8M4.txt,34.0905,32.6383,22.9954,9.6428,29.545,1.45228,117.908,12.317,33.005
2,Batch_5,M0,Pre_Scan,Male,B8M0.txt,40.0617,38.7079,28.1945,10.5133,27.161,1.35384,106.786,12.678,37.423
3,Batch_5,M1,Pre_Scan,Male,B8M1.txt,39.6344,38.4868,30.1166,8.3702,21.748,1.14761,103.150,11.126,36.142
4,Batch_5,M3,Pre_Scan,Male,B8M3.txt,32.9502,31.8330,22.5268,9.3062,29.234,1.11723,103.480,10.797,32.174


## Export Results

In [7]:
# Export to Excel with multiple sheets
excel_output_path = output_dir / "batch5_dexa_cleaned.xlsx"

with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
    # Main data sheet
    batch5_cleaned.to_excel(writer, sheet_name='Batch5_All_Data', index=False)
    
    # Summary by timepoint
    timepoint_summary = batch5_cleaned.groupby(['timepoint', 'gender']).agg({
        'subject_id': 'nunique',
        'total_weight': 'mean',
        'fat_percent': 'mean',
        'bmd': 'mean',
        'lean_weight': 'mean'
    }).round(3)
    timepoint_summary.to_excel(writer, sheet_name='Timepoint_Summary')
    
    # Subject tracking sheet
    subject_summary = batch5_cleaned.groupby('subject_id').agg({
        'timepoint': 'nunique',
        'gender': 'first',
        'total_weight': ['min', 'max', 'mean'],
        'fat_percent': ['min', 'max', 'mean']
    }).round(3)
    subject_summary.columns = ['_'.join(col).strip() for col in subject_summary.columns]
    subject_summary.to_excel(writer, sheet_name='Subject_Summary')

print(f"Excel file exported: {excel_output_path}")
print(f"Sheets created: Batch5_All_Data, Timepoint_Summary, Subject_Summary")

# Also save as CSV for backup
csv_output_path = output_dir / "batch5_dexa_cleaned.csv"
batch5_cleaned.to_csv(csv_output_path, index=False)
print(f"CSV backup saved: {csv_output_path}")

print(f"\nFinal Results Summary:")
print(f"- Total records: {len(batch5_cleaned)}")
print(f"- Unique subjects: {batch5_cleaned['subject_id'].nunique()}")
print(f"- Timepoints: {list(batch5_cleaned['timepoint'].unique())}")
print(f"- Gender distribution: {batch5_cleaned['gender'].value_counts().to_dict()}")
print(f"- Key measurements: total_weight, fat_percent, bmd, lean_weight, fat_weight")

Excel file exported: ../../../cleaned_output/batch5_dexa_cleaned.xlsx
Sheets created: Batch5_All_Data, Timepoint_Summary, Subject_Summary
CSV backup saved: ../../../cleaned_output/batch5_dexa_cleaned.csv

Final Results Summary:
- Total records: 50
- Unique subjects: 10
- Timepoints: ['Pre_Scan', 'Week_1', 'Week_2', 'Week_3', 'Post_Scan']
- Gender distribution: {'Male': 30, 'Female': 20}
- Key measurements: total_weight, fat_percent, bmd, lean_weight, fat_weight
